<div class="alert alert-block alert-success"> 
    
# How to use this notebook
    
This notebook can perform all the steps required for data analysis of SELEX-seq data with the SELEX package. To use it, follow these steps: 

### 1. Input variables
All file paths, file names and as well as information such as the variable length need to be **given as input variables** in the section indicated below. At the very end of this notebook there is a blue box giving detailed information on how to download and save data sets from the Gene Expression Omnibus, how to interpret the FASTQ files and how to save time by saving the metadata in an annotation file.
    
### 2. Setting up SELEX
The next steps require **no input - just run the cells**. Appropriate settings are set and the data is imported properly such that the SELEX package 'knows' how to deal with and address the data you have provided. 
    
### 3. Analysis
This step requires one input - indicate the round that is to be compared to R0 for the calculation of the information gain and affinities.
    
</div>

<div class="alert alert-block alert-warning"> 

In the following, all sections requiring input are highlighted in yellow.
    
</div>

<div class="alert alert-block alert-warning">

# 1. Input Variables
    
### Protein Name & Round names & Sample Name
Indicate the name of the protein being analyzed, as a string. We will use this string to name samples and output data. Likewise, indicate the name of the sample sequences the protein is analyzed with.

</div>

In [ ]:
workdir="/home/jmuino/scratch/FlowerDevGroup/Peilin/TestJupyter"           # working directory
pname = 'FUL'                                        # name of the TF/protein to be analyzed

 <div class="alert alert-block alert-warning">

### File paths
    

The program neeed the filepath for the fasta/fastq files containing the seqeuncing of the DNA oligos for round 0 (r0_filepath) and one round of enrichment (r_filepath). The parameter round_en indicates which round is used in r_filepath (eg: round 4 of enrichment)
    </div>

In [ ]:
r0_filepath = 'data/FUL-FUL-R0_test.fastq.gz'
r_filepath = 'data/FUL-FUL-R4_test.fastq.gz'
round_en = 4   # it should be an integer

<div class="alert alert-block alert-warning">

### Variable length, Left and Right Barcodes
This is the length of the variable region of the nucleic acid snippets in the sample. If sequences are purely random with no defined left or right barcodes, the length of the variable region is equal to the length of the sequence.
    
</div>

In [ ]:
variable_length = 20
left_bc = ''
right_bc = ''

# 2.a) Required settings for SELEX

In [ ]:
setwd(workdir)
suppressMessages(require(SELEX))
options(java.parameters="-Xmx2500M")                # Set maximum memory usage limit
tempDir = "./cache/"                                # Specify the temporal working directory
unlink(file.path(path.expand("~"), "SELEX.properties"))
selex.config(workingDir=tempDir, maxThreadNumber=4) # Configure Selex with 4 maxThreadNumber


The working directory contains the temporary files. The folder is created if it does not exist, everything in it is automatically kept track of and does not need to be read or addressed by a person. Once calculation is complete, the contents or even the whole folder can safely be deleted. 

# 2.b) Import your own data

Detailed instructions for downloading data that is available on the GEO are given below, in the last cell in this notebook.

#### Define metadata

This step is required to save metadata such as the filepath, barcodes and length of the variable region. This information is necessary for SELEX to interpret the data in subsequent steps. 

In [ ]:
rname=paste("R",round_en,sep="")
sample_r0 = selex.defineSample(seqName = paste(pname, 'R0', sep='-'), seqFile = r0_filepath, 
                               sampleName = paste('R0', pname, sep='.'), round = 0, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

sample_r = selex.defineSample(seqName = paste(pname, rname, sep='-'), seqFile = r_filepath, 
                               sampleName = paste(rname, pname, sep='.'), round = round_en, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)


#### Import the samples

In [ ]:
r0 = selex.sample(seqName = paste(pname, 'R0', sep='-'), sampleName = paste('R0', pname, sep='.'), round = 0)

Round2BAnalyzed = selex.sample(seqName = paste(pname, rname, sep='-'), sampleName = paste(rname, pname, sep='.'), round = round_en)


#### Split the first sample into a training and a testing part

Default ratio is 0.5 (i.e. half training and half testing data). 

In [ ]:
r0.split = selex.split(r0, ratio=c(0.5,0.5))

#### View imported data

In [ ]:
selex.sampleSummary()

# 3. Analysis



## Markov model

Even though random sequences of DNA are used, the frequency with which they occur is not completely free of bias - i.e. not all random sequences are equally likely to be found.

The SELEX method uses a Markov model to predict the number of occurences of each possible k-mer in round zero (R0).

### 3.1 Count $k_{max}$

Find the maximal nucleotide length $k_{max}$ such that all sequences of that length $k_{max}$ appear at least $n$ times in the data, where $n$ is the threshold value (default: 100). 

The model will be fitted to sequences of length $k_{max}$, all other sequences will be neglected for the purpose of fitting the Markov Model. Therefore, it is important that $n$ be large enough to allow for a statistically sound fit. 

In [ ]:
# Find the longest length k such that all possible k-mers are found at least 100 times
kmax.value = selex.kmax(sample=r0.split$test, threshold=100, seqfilter=NULL)

In [ ]:
#Kmax.value is the value of the adviced Kmer length
kmax.value

### 3.2 Create and store the Markov model

Next, training and testing data as well as $k_{max}$ is passed to SELEX which creates and saves a Markov model. If the order is not specified, it will test Markov models of increasing order, selecting the optimal order.

The order of the Markov model refers to the number of nucleotides considered when inferring the conditional occurence probabilities of subsequent nucleotides, e.g. P(A|GATTC) for a Markov model of the fifth order.

In [ ]:
# Requires samples to be used for training and testing, as well as the k_max value.
mm = selex.mm(sample=r0.split$train, order=NA, crossValidationSample=r0.split$test, Kmax=kmax.value)

In [ ]:
selex.mmSummary()

### 3.3 Information gain and optimal length

The last line printed is the maximal information gain. 

<div class="alert alert-block alert-danger">
    
If the following line throws **this error**: 
    
>Error in .jcall("RJavaTools", "Ljava/lang/Object;", "invokeMethod", cl, : java.lang.OutOfMemoryError: Java heap space
    
It helps to delete the whole cache (working directory) and then restart the notebook (using the >> Button in the menu). 

</div>

In [ ]:
selex.infogain(Round2BAnalyzed, markovModel=mm)

Check which value of k corresponds to the maximal information gain: 

In [ ]:
sgain<-selex.infogainSummary();
sgain

<div class="alert alert-block alert-warning">

The **optimal length** is the value of k for which the information gain is highest. Save that information in the variable below:
    
</div>

In [ ]:
optimalLength <- sgain$K[sgain$InformationGain==max(sgain$InformationGain)][1]

Count how many times k-mers of optimal length appear: 

In [ ]:
table = selex.counts(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 
head(table)

### Calculating affinity and error

Affinities are calculated for specific rounds, so the sample needs to be specified appropriately (i.e. enter r2 for round 2 or r3 for round 3, etc.)

In [ ]:
aff = selex.affinities(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 

In [ ]:
head(aff)

### Save and export the affinities table

In [ ]:
export_table_name = paste(paste('', pname, sep=''), rname, 'affinities.csv', sep='_')
write.csv(aff, export_table_name)


# Resources

<div class="alert alert-block alert-info"><b>Credit:</b> This notebook is adapted from the SELEX R package described in [Paper describing the SELEX-seq anaysis method][selex paper]: Cofactor binding evokes latent differences in DNA binding specificity between Hox proteins, Slattery et al., 2011.
    
[SELEX R package Manual][selex manual]</div>





[selex example]: https://bioconductor.org/packages/release/bioc/vignettes/SELEX/inst/doc/SELEX.pdf
[selex manual]: https://bioconductor.org/packages/release/bioc/manuals/SELEX/man/SELEX.pdf
[selex paper]: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3319069/